In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd 
import os
import time
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from model import lstm, aagcn, stconv, SAM
from data.handpose_dataset import HandPoseDatasetNumpy, df_to_numpy
from data.get_data_from_csv import get_train_data, get_val_data
from config import CFG
from utils import training_supervision, adj_mat

In [32]:
device = torch.device("cuda")

In [41]:
df = pd.read_csv("data/graphdata/custom_data/test_big_2.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

,WRIST,THUMB_CMC,THUMB_MCP,THUMB_IP,THUMB_TIP,INDEX_FINGER_MCP,INDEX_FINGER_PIP,INDEX_FINGER_DIP,INDEX_FINGER_TIP,MIDDLE_FINGER_MCP,...,MIDDLE_FINGER_TIP,RING_FINGER_MCP,RING_FINGER_PIP,RING_FINGER_DIP,RING_FINGER_TIP,PINKY_MCP,PINKY_PIP,PINKY_DIP,PINKY_TIP,LABEL
0,"(0.53, 1.23, 0.14)","(0.50, 1.23, 0.13)","(0.46, 1.21, 0.15)","(0.44, 1.20, 0.17)","(0.43, 1.19, 0.18)","(0.45, 1.24, 0.19)","(0.43, 1.23, 0.21)","(0.41, 1.22, 0.23)","(0.40, 1.22, 0.24)","(0.46, 1.24, 0.20)",...,"(0.42, 1.21, 0.27)","(0.48, 1.23, 0.21)","(0.46, 1.22, 0.24)","(0.46, 1.22, 0.26)","(0.45, 1.21, 0.27)","(0.50, 1.23, 0.21)","(0.49, 1.22, 0.23)","(0.48, 1.21, 0.25)","(0.48, 1.20, 0.25)",Grasp
1,"(0.53, 1.24, 0.14)","(0.50, 1.23, 0.14)","(0.46, 1.21, 0.15)","(0.44, 1.20, 0.17)","(0.43, 1.19, 0.18)","(0.45, 1.24, 0.19)","(0.43, 1.23, 0.21)","(0.41, 1.22, 0.23)","(0.40, 1.22, 0.24)","(0.46, 1.24, 0.20)",...,"(0.42, 1.21, 0.27)","(0.48, 1.23, 0.21)","(0.46, 1.22, 0.24)","(0.46, 1.22, 0.26)","(0.45, 1.21, 0.27)","(0.50, 1.23, 0.21)","(0.49, 1.22, 0.23)","(0.48, 1.21, 0.25)","(0.48, 1.20, 0.25)",Grasp
2,"(0.53, 1.23, 0.14)","(0.50, 1.23, 0.14)","(0.46, 1.21, 0.15)","(0.44, 1.20, 0.17)","(0.43, 1.19, 0.18)","(0.45, 1.24, 0.19)","(0.43, 1.23, 0.21)","(0.41, 1.22, 0.23)","(0.40, 1.22, 0.24)","(0.46, 1.24, 0.20)",...,"(0.42, 1.21, 0.27)","(0.48, 1.23, 0.21)","(0.47, 1.22, 0.24)","(0.46, 1.22, 0.26)","(0.45, 1.21, 0.27)","(0.50, 1.23, 0.21)","(0.49, 1.22, 0.23)","(0.48, 1.21, 0.25)","(0.48, 1.20, 0.25)",Grasp
3,"(0.53, 1.23, 0.14)","(0.50, 1.23, 0.13)","(0.46, 1.21, 0.15)","(0.44, 1.20, 0.17)","(0.43, 1.19, 0.18)","(0.45, 1.24, 0.19)","(0.43, 1.23, 0.21)","(0.41, 1.22, 0.23)","(0.40, 1.22, 0.24)","(0.46, 1.24, 0.20)",...,"(0.42, 1.21, 0.27)","(0.48, 1.23, 0.21)","(0.47, 1.22, 0.24)","(0.46, 1.22, 0.26)","(0.45, 1.21, 0.27)","(0.50, 1.23, 0.21)","(0.49, 1.22, 0.23)","(0.48, 1.21, 0.25)","(0.48, 1.20, 0.25)",Grasp
4,"(0.53, 1.23, 0.14)","(0.50, 1.23, 0.14)","(0.46, 1.21, 0.15)","(0.44, 1.20, 0.17)","(0.43, 1.19, 0.18)","(0.45, 1.24, 0.19)","(0.43, 1.23, 0.21)","(0.41, 1.22, 0.23)","(0.40, 1.22, 0.24)","(0.46, 1.24, 0.20)",...,"(0.42, 1.21, 0.27)","(0.48, 1.23, 0.21)","(0.46, 1.22, 0.24)","(0.46, 1.22, 0.26)","(0.45, 1.21, 0.27)","(0.50, 1.23, 0.21)","(0.49, 1.22, 0.23)","(0.48, 1.21, 0.25)","(0.48, 1.20, 0.25)",Grasp


In [42]:
data_numpy = df_to_numpy(df)
data_numpy

(array([[0.53, 1.23, 0.14, ..., 0.48, 1.2 , 0.25],
        [0.53, 1.24, 0.14, ..., 0.48, 1.2 , 0.25],
        [0.53, 1.23, 0.14, ..., 0.48, 1.2 , 0.25],
        ...,
        [0.59, 0.95, 0.24, ..., 0.58, 0.96, 0.37],
        [0.59, 0.95, 0.24, ..., 0.58, 0.96, 0.37],
        [0.59, 0.95, 0.24, ..., 0.57, 0.96, 0.37]]),
 array([[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]]))

In [43]:
test_set_2 = HandPoseDatasetNumpy(data_numpy, distances=True)
test_loader_2 = DataLoader(test_set_2, batch_size=CFG.batch_size, drop_last=True)
graph = aagcn.Graph(adj_mat.num_node, adj_mat.self_link, adj_mat.inward, adj_mat.outward, adj_mat.neighbor)
model_2 = aagcn.Model(num_class=CFG.num_classes, num_point=21, num_person=1, graph=graph, drop_out=0.5, in_channels=3)
print(f"[INFO] TESTING ON {len(test_set_2)} DATAPOINTS")

[INFO] TESTING ON 4160 DATAPOINTS


In [44]:
class FocalLoss(nn.Module):
    
    def __init__(self, weight=None, 
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob, 
            target_tensor, 
            weight=self.weight,
            reduction = self.reduction
        )

In [45]:
def eval_func(model, criterion, data_loader, epoch):
    model.eval()
    preds = []
    groundtruth = []
    t0 = time.time()
    loss_total = 0
    global_step = 0
    iters = len(data_loader)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(data_loader):
            labels = labels.cuda().long()
            inputs = inputs.cuda().float()

            last_label = labels[:, -1, :]
            last_label = torch.argmax(last_label, 1)

            last_out = model(inputs)
            loss = criterion(last_out, last_label)

            preds.append(last_out.cpu().detach().numpy())
            groundtruth.append(last_label.cpu().detach().numpy())
            loss_total += loss

            if i%CFG.print_freq == 1 or i == iters-1:
                t1 = time.time()
                print(f"[EVAL] Iteration: {i}/{iters} | Val-Loss: {loss_total/i} | ETA: {((t1-t0)/i * iters) - (t1-t0)}s")

    return loss_total, np.array(preds),  np.array(groundtruth).flatten()

In [49]:
MODEL_PATH_2 = "trained_models/7_AAGCN_Focal_seqlen32_release_SAM_joints1_joints2_oridist/f10.8142688679245284_valloss310.2437744140625_epoch13.pth"

#MODEL_PATH_2 = "fine_tuned_models_occluded_hand_detection/7AAGCN_seqlen32_finetuned_SAM_joints1_joints2_/f10.3802083333333333_valloss29.320728302001953_epoch3.pth"


In [50]:
model_2.load_state_dict(torch.load(MODEL_PATH_2)["model_state_dict"])
model_2.cuda()

criterion = FocalLoss()

In [51]:
val_loss_2, preds_val_2, gt_val_2 = eval_func(model_2, criterion, test_loader_2, 0)
print(f"[EVAL] VALIDATION LOSS MODEL 2 {val_loss_2}")
print(classification_report(gt_val_2, np.argmax(preds_val_2, axis=2).flatten(), target_names=CFG.classes, digits=4))


[EVAL] Iteration: 1/260 | Val-Loss: 2.315990924835205 | ETA: 64.76926612854004s
[EVAL] Iteration: 101/260 | Val-Loss: 1.319405436515808 | ETA: 5.703596060818963s
[EVAL] Iteration: 201/260 | Val-Loss: 1.3154160976409912 | ETA: 2.020185350778684s
[EVAL] Iteration: 259/260 | Val-Loss: 1.7616090774536133 | ETA: 0.03384550105650952s
[EVAL] VALIDATION LOSS MODEL 2 456.25677490234375
              precision    recall  f1-score   support

       Grasp     0.4000    0.1833    0.2514       993
        Move     0.4286    0.3998    0.4137      1088
    Negative     0.2107    1.0000    0.3480       288
    Position     0.2781    0.2917    0.2847       576
       Reach     0.2025    0.2250    0.2132       640
     Release     0.0000    0.0000    0.0000       575

    accuracy                         0.2925      4160
   macro avg     0.2533    0.3500    0.2518      4160
weighted avg     0.2918    0.2925    0.2645      4160
